In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


import pandas as pd
import numpy as np
from collections import Counter

n_children = 1000000 # n children to give
n_gift_type = 1000 # n types of gifts available
n_gift_quantity = 1000 # each type of gifts are limited to this quantity
n_gift_pref = 10 # number of gifts a child ranks
n_child_pref = 1000 # number of children a gift ranks
twins = int(0.004 * n_children)    # 0.4% of all population, rounded to the closest even number
ratio_gift_happiness = 2
ratio_child_happiness = 2


gift_pref = pd.read_csv('../input/child_wishlist.csv',header=None).drop(0, 1).values
child_pref = pd.read_csv('../input/gift_goodkids.csv',header=None).drop(0, 1).values


def avg_normalized_happiness(pred, child_pref, gift_pref):
    
    # check if number of each gift exceeds n_gift_quantity
    gift_counts = Counter(elem[1] for elem in pred)
    for count in gift_counts.values():
        assert count <= n_gift_quantity
                
    # check if twins have the same gift
    for t1 in range(0,twins,2):
        twin1 = pred[t1]
        twin2 = pred[t1+1]
        assert twin1[1] == twin2[1]
    
    max_child_happiness = n_gift_pref * ratio_child_happiness
    max_gift_happiness = n_child_pref * ratio_gift_happiness
    total_child_happiness = 0
    total_gift_happiness = np.zeros(n_gift_type)
    
    for row in pred:
        child_id = row[0]
        gift_id = row[1]
        
        # check if child_id and gift_id exist
        assert child_id < n_children
        assert gift_id < n_gift_type
        assert child_id >= 0 
        assert gift_id >= 0
        child_happiness = (n_gift_pref - np.where(gift_pref[child_id]==gift_id)[0]) * ratio_child_happiness
        if not child_happiness:
            child_happiness = -1

        gift_happiness = ( n_child_pref - np.where(child_pref[gift_id]==child_id)[0]) * ratio_gift_happiness
        if not gift_happiness:
            gift_happiness = -1

        total_child_happiness += child_happiness
        total_gift_happiness[gift_id] += gift_happiness
    
    # print(max_child_happiness, max_gift_happiness
    print('normalized child happiness=',float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) , \
        ', normalized gift happiness',np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity))
    return float(total_child_happiness)/(float(n_children)*float(max_child_happiness)) + np.mean(total_gift_happiness) / float(max_gift_happiness*n_gift_quantity)

random_sub = pd.read_csv('../input/sample_submission_random.csv').values.tolist()
print(avg_normalized_happiness(random_sub, child_pref, gift_pref))

child_wishlist.csv
gift_goodkids.csv
sample_submission_random.csv

normalized child happiness= -0.04404195 , normalized gift happiness 1.18475e-05
-0.0440301025


In [2]:
gift_pref.shape, child_pref.shape

((1000000, 10), (1000, 1000))

In [3]:
class Child(object):
    
    def __init__(self, idx, prefer):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for i in range(prefer.shape[0]):
            self.prefer_dict[prefer[i]] = 400*(prefer.shape[0] - i) - 2
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid] += 2*score + 2
        else:
            self.prefer_dict[giftid] = 2*score - 200
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, -202)

In [4]:
class Child_twin(object):
    
    def __init__(self, idx, prefer1, prefer2):
        
        self.idx = idx
        self.prefer_dict = dict()
        
        for p in list(set(list(prefer1) + list(prefer2))):
            score = 0
            if p in list(prefer1):
                score += 2*(10 - list(prefer1).index(p))
            else:
                score -= 1
            if p in list(prefer2):
                score += 2*(10 - list(prefer2).index(p))
            else:
                score -= 1
            self.prefer_dict[p] = 100*score - 2
    
    
    def add_gifts_prefer(self, giftid, score):
        
        if giftid in self.prefer_dict.keys():
            self.prefer_dict[giftid] += score + 2
        else:
            self.prefer_dict[giftid] = score - 200
        
        return None
        
    
    def happiness(self, giftid):
        
        return self.prefer_dict.get(giftid, -202)

In [5]:
Children = []
for i in range(2000):
    Children.append(Child_twin(2*i, gift_pref[2*i], gift_pref[2*i+1]))
    Children.append(Child_twin(2*i+1, gift_pref[2*i], gift_pref[2*i+1]))
Children = Children + [Child(i, gift_pref[i]) for i in range(4000, 1000000)]

In [6]:
for j in range(1000):
    cf = child_pref[j]
    done_list = []
    for i in range(cf.shape[0]):
        if cf[i] < 4000 and cf[i] not in done_list:
            if cf[i] % 2 == 0:
                cid1 = cf[i]
                cid2 = cf[i] + 1
                done_list.append(cid2)
            else:
                cid1 = cf[i] - 1
                cid2 = cf[i]
                done_list.append(cid1)
            if cid1 in list(cf):
                score_ = 2*(cf.shape[0] - list(cf).index(cid1))
            else:
                score_ = -1
            if cid2 in list(cf):
                score_ += 2*(cf.shape[0] - list(cf).index(cid2))
            else:
                score_ += -1
            Children[cid1].add_gifts_prefer(j, score_)
            Children[cid2].add_gifts_prefer(j, score_)
        elif cf[i] >= 4000:
            Children[cf[i]].add_gifts_prefer(j, 2*(cf.shape[0] - i))

In [7]:
start_nodes = []
end_nodes = []
capacities = []
unit_costs = []


# twins
for i in range(2000):
    if i in [396, 1222, 1802]:
        pass
    else:
        for g in Children[2*i].prefer_dict.keys():
            if g == 337:
                pass
            else:
                start_nodes.append(g)
                end_nodes.append(i+1000)
                capacities.append(2)
                unit_costs.append(-(Children[2*i].prefer_dict[g] + 202))
        
# non twins
for i in range(4000, 1000000):
    
    for g in Children[i].prefer_dict.keys():
        start_nodes.append(g)
        end_nodes.append(i-1000)
        capacities.append(1)
        unit_costs.append(-(Children[i].prefer_dict[g] + 202))






In [8]:
from ortools.graph import pywrapgraph

In [9]:
min_cost_flow_2 = pywrapgraph.SimpleMinCostFlow()
# add Arc
for i in range(len(start_nodes)):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(unit_costs[i])
    )
for i in range(1000, 3000):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(i), int(999000), int(2), int(0)
    )
for i in range(4000, 1000000):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(i-1000), int(999000), int(1), int(0)
    )
for i in range(1000):
    min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(i), int(999001), int(1000), int(0)
    )
min_cost_flow_2.AddArcWithCapacityAndUnitCost(
        int(999001), int(999000), int(1000000), int(0)
    )
# add Supply
for i in range(1000):
    if i == 337:
        min_cost_flow_2.SetNodeSupply(int(i), int(1000 - 6))
    else:
        min_cost_flow_2.SetNodeSupply(int(i), int(1000))
# twins
for i in range(1000, 3000):
    min_cost_flow_2.SetNodeSupply(int(i), int(0))
for i in range(4000, 1000000):
    min_cost_flow_2.SetNodeSupply(int(i-1000), int(0))  

min_cost_flow_2.SetNodeSupply(int(999001), int(0)) 
min_cost_flow_2.SetNodeSupply(int(999000), int(-1000000 + 6)) 

min_cost_flow_2.Solve()

NameError: name 'rand_cut' is not defined

In [17]:
print(min_cost_flow_2.OptimalCost() - (1898+202)*6)

-3972156060


In [14]:
Children[792].happiness(337), Children[2444].happiness(337), Children[3604].happiness(337)

(1898, 1898, 1898)

In [30]:
-3972156062 

-3972156062

In [18]:
assignment = [-1]*1000000
twins_differ = []

for i in range(min_cost_flow_2.NumArcs()):
    if min_cost_flow_2.Flow(i) != 0 and min_cost_flow_2.Head(i) < 999000:
        c = min_cost_flow_2.Head(i)
        g = min_cost_flow_2.Tail(i)

        if c >= 3000:
            assignment[c+1000] = g

        elif min_cost_flow_2.Flow(i) == 1:
            twins_differ.append([2*(c - 1000), 2*(c - 1000) + 1, assignment[2*(c - 1000)], g])
            if assignment[2*(c - 1000)] != -1:
                assignment[2*(c - 1000) + 1] = g
            else:
                assignment[2*(c - 1000)] = g

        elif min_cost_flow_2.Flow(i) == 2:
            assignment[2*(c - 1000)] = g
            assignment[2*(c - 1000) + 1] = g

Gifts_left = [1000 for _ in range(1000)]
odd_gifts = []
for i in range(1000000):
    if assignment[i] != -1:
        Gifts_left[assignment[i]] -= 1
for i in range(1000):
    if Gifts_left[i] % 2 == 1:
        odd_gifts.append([i, Gifts_left[i]])

unhappy_nontwin = []
for i in range(4000, 1000000):
    if assignment[i] == -1:
        unhappy_nontwin.append(i)
print(twins_differ, odd_gifts, unhappy_nontwin)

[] [[108, 243], [319, 155], [349, 227], [429, 157], [438, 51], [469, 229], [473, 89], [758, 179]] []


In [23]:
assignment[792] = 337
assignment[793] = 337
assignment[2444] = 337
assignment[2445] = 337
assignment[3604] = 337
assignment[3605] = 337

In [43]:
g_target = 758

G_i = dict()
G_u = dict()
for j in range(1000):
    G_i[j] = dict()
for i in range(4000, 1000000):
    j = assignment[i]
    G_u[i] = [j, Children[i].happiness(j)]
    for k in Children[i].prefer_dict.keys():
        if k != j:
            G_i[k][i] = - Children[i].happiness(k)


G_u[1001] = [g_target, -202]
for j in range(1000):
    if j != g_target:
        G_i[j][1001] = 202

heat_u = [100000]*1000000
heat_i = [100000]*1000
path_u = [-1]*1000000
path_i = [-1]*1000
heat_u_active_list = [1001]
heat_u[1001] = 0


while len(heat_u_active_list) > 0:
    # Loop
    heat_i_active_list = []
    for u in heat_u_active_list:
        new_i, cost = G_u[u]
        if heat_u[u] + cost < heat_i[new_i]: 
            heat_i[new_i] = heat_u[u] + cost
            heat_i_active_list.append(new_i)
            path_i[new_i] = u
    heat_u_active_list = []

    # Loop
    for i in heat_i_active_list:
        for u in G_i[i].keys():
            if u == 1001:
                if heat_i[i] + G_i[i][u] == 0:
                    path_u[u] = i
                    print("Load")
                elif heat_i[i] + G_i[i][u] < 0:
                    path_u[u] = i
                    print("Was not Optimal!!")
            elif heat_i[i] + G_i[i][u] < heat_u[u]: 
                heat_u[u] = heat_i[i] + G_i[i][u]
                heat_u_active_list.append(u)
                path_u[u] = i

    # 202 Fire
    fire = min(heat_i) + 202
    fire_source = np.argmin(heat_i)
    heat_i_ = heat_i.copy()
    heat_i_[g_target] = 1000000
    fire_ = min(heat_i_) + 202
    fire_source_ = np.argmin(heat_i_)
    if fire_ == 0:
        print("Fire")
        path_u[1001] = fire_source_
    for u in range(4000, 1000000):
        if fire < heat_u[u]: 
            heat_u[u] = fire
            heat_u_active_list.append(u)
            path_u[u] = fire_source
    heat_u_active_list = list(np.unique(heat_u_active_list))

In [ ]:
[[108, 243], [319, 155], [349, 227], [429, 157], [438, 51], [469, 229], [473, 89], [758, 179]]

In [20]:
res_108 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [25]:
res_319 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [28]:
res_349 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [32]:
res_429 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [35]:
res_438 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [38]:
res_469 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [41]:
res_473 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [44]:
res_758 = [heat_u.copy(), heat_i.copy(), path_u.copy(), path_i.copy()]

In [22]:
[
    res_108[1][108], res_108[1][319], res_108[1][349], res_108[1][429], 
    res_108[1][438], res_108[1][469], res_108[1][473], res_108[1][758]
]

[-202, -198, -200, -198, -200, -186, -200, -194]

In [26]:
[
    res_319[1][108], res_319[1][319], res_319[1][349], res_319[1][429], 
    res_319[1][438], res_319[1][469], res_319[1][473], res_319[1][758]
]

[-194, -202, -200, -198, -200, -186, -200, -194]

In [29]:
[
    res_349[1][108], res_349[1][319], res_349[1][349], res_349[1][429], 
    res_349[1][438], res_349[1][469], res_349[1][473], res_349[1][758]
]

[-186, -190, -202, -190, -192, -178, -192, -186]

In [33]:
[
    res_429[1][108], res_429[1][319], res_429[1][349], res_429[1][429], 
    res_429[1][438], res_429[1][469], res_429[1][473], res_429[1][758]
]

[-194, -198, -200, -202, -200, -186, -200, -194]

In [36]:
[
    res_438[1][108], res_438[1][319], res_438[1][349], res_438[1][429], 
    res_438[1][438], res_438[1][469], res_438[1][473], res_438[1][758]
]

[-194, -198, -200, -198, -202, -186, -200, -194]

In [39]:
[
    res_469[1][108], res_469[1][319], res_469[1][349], res_469[1][429], 
    res_469[1][438], res_469[1][469], res_469[1][473], res_469[1][758]
]

[-196, -200, -202, -200, -202, -202, -202, -196]

In [42]:
[
    res_473[1][108], res_473[1][319], res_473[1][349], res_473[1][429], 
    res_473[1][438], res_473[1][469], res_473[1][473], res_473[1][758]
]

[-190, -194, -196, -194, -196, -182, -202, -190]

In [45]:
[
    res_758[1][108], res_758[1][319], res_758[1][349], res_758[1][429], 
    res_758[1][438], res_758[1][469], res_758[1][473], res_758[1][758]
]

[-192, -200, -198, -196, -198, -184, -198, -202]

In [56]:
# 758 -> 319
# cost 2

path_u = res_758[2].copy()
path_i = res_758[3].copy()


i_ = 319

u = path_i[i_]
res_list = [i_, u]
while True:
    i = path_u[u]
    if i == 758:
        res_list.append(i)
        break
    u = path_i[i]
    res_list.append(i)
    res_list.append(u)



In [58]:
res_list_1 = res_list.copy()
res_list_1

[319, 263572, 563, 82231, 763, 615584, 758]

In [66]:
len(res_list_1) // 2

3

In [59]:
# 469 -> 473
# cost 0

path_u = res_469[2].copy()
path_i = res_469[3].copy()


i_ = 473

u = path_i[i_]
res_list = [i_, u]
while True:
    i = path_u[u]
    if i == 469:
        res_list.append(i)
        break
    u = path_i[i]
    res_list.append(i)
    res_list.append(u)

In [60]:
res_list_2 = res_list.copy()
res_list_2

[473,
 643959,
 808,
 118165,
 925,
 42674,
 136,
 253049,
 914,
 808473,
 633,
 278142,
 469]

In [61]:
# 108 -> 438
# cost 2

path_u = res_108[2].copy()
path_i = res_108[3].copy()


i_ = 438

u = path_i[i_]
res_list = [i_, u]
while True:
    i = path_u[u]
    if i == 108:
        res_list.append(i)
        break
    u = path_i[i]
    res_list.append(i)
    res_list.append(u)

In [62]:
res_list_3 = res_list.copy()
res_list_3

[438, 507661, 661, 464041, 970, 234600, 317, 35718, 380, 628503, 108]

In [63]:
# 429 -> 349
# cost 2

path_u = res_429[2].copy()
path_i = res_429[3].copy()


i_ = 349

u = path_i[i_]
res_list = [i_, u]
while True:
    i = path_u[u]
    if i == 429:
        res_list.append(i)
        break
    u = path_i[i]
    res_list.append(i)
    res_list.append(u)

In [64]:
res_list_4 = res_list.copy()
res_list_4

[349, 938606, 234, 579653, 106, 789486, 62, 678603, 429]

In [65]:
assignment_ = assignment.copy()

In [67]:
# switch
for i in range(len(res_list_1) // 2):
    assignment_[res_list_1[2*i+1]] = res_list_1[2*i+2]
for i in range(len(res_list_2) // 2):
    assignment_[res_list_2[2*i+1]] = res_list_2[2*i+2]
for i in range(len(res_list_3) // 2):
    assignment_[res_list_3[2*i+1]] = res_list_3[2*i+2]
for i in range(len(res_list_4) // 2):
    assignment_[res_list_4[2*i+1]] = res_list_4[2*i+2]

In [68]:
sum([Children[i].happiness(assignment_[i]) for i in range(1000000)])

3770156054

In [69]:
3770156054 / 4000000000

0.9425390135

In [72]:
Gifts_left = [1000 for _ in range(1000)]
odd_gifts = []
for i in range(1000000):
    if assignment_[i] != -1:
        Gifts_left[assignment_[i]] -= 1
for i in range(1000):
    if Gifts_left[i] % 2 == 1:
        odd_gifts.append([i, Gifts_left[i]])

In [73]:
odd_gifts

[]

In [74]:
left_gifts = []
for j in range(1000):
    for _ in range(Gifts_left[j]):
        left_gifts.append(j)

In [75]:
len(left_gifts), sum(Gifts_left)

(3024, 3024)

In [76]:
# assign for twins
for i in range(1000000):
    if assignment_[i] == -1:
        assignment_[i] = left_gifts.pop()

In [78]:
avg_normalized_happiness(res_list_temp, child_pref, gift_pref)

normalized child happiness= 0.8704815 , normalized gift happiness 0.0720575135


0.94253901350000002

In [77]:
res_list_temp = [[i, assignment_[i]] for i in range(1000000)]
out = open('../submit/subm_hrd_final.csv', 'w')
out.write('ChildId,GiftId\n')
for i in range(len(res_list_temp)):
    out.write(str(i) + ',' + str(res_list_temp[i][1]) + '\n')
out.close()